In [1]:
import pyspark
from pyspark import SQLContext
from pyspark import SparkContext
import time
from pyspark.sql import Row
from pyspark.sql.functions import lag, col, toRadians, sqrt, sin, cos, asin, lit, unix_timestamp, from_unixtime, datediff,sum
from pyspark.sql.functions import isnan, when, count, col, expr
from pyspark.sql.window import Window
from pyspark import SparkConf
from pyspark.sql.functions import countDistinct
import numpy as np

In [2]:
#sc = pyspark.SparkContext.getOrCreate()
#https://datascience.stackexchange.com/questions/8549/how-do-i-set-get-heap-size-for-spark-via-python-notebook

conf = SparkConf().setAppName("App")
conf = (conf.setMaster('local[*]')
        .set('spark.executor.memory', '4G')
        .set('spark.driver.memory', '45G')
        .set('spark.driver.maxResultSize', '10G'))
sc = SparkContext(conf=conf)
sc

<SparkContext master=local[*] appName=App>

In [3]:
sqlContext = SQLContext(sc)

In [73]:
## Loading csv file as the RDD
HappinessdataSetRDD= sc.textFile("2019.csv")

In [5]:
## to find the type of the object
type(HappinessdataSetRDD)

pyspark.rdd.RDD

In [6]:
## looking for top 10 rows using take 
HappinessdataSetRDD.take(10)

['1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393',
 '2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410',
 '3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341',
 '4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118',
 '5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298',
 '6,Switzerland,7.480,1.452,1.526,1.052,0.572,0.263,0.343',
 '7,Sweden,7.343,1.387,1.487,1.009,0.574,0.267,0.373',
 '8,New Zealand,7.307,1.303,1.557,1.026,0.585,0.330,0.380',
 '9,Canada,7.278,1.365,1.505,1.039,0.584,0.285,0.308',
 '10,Austria,7.246,1.376,1.475,1.016,0.532,0.244,0.226']

In [7]:
## mapping that data using required columns as from the RDD using map function
HappinessdataSetRDDCountryScore= HappinessdataSetRDD.map(lambda x: ((int(x.split(",")[0])), x.split(",")[1], float(x.split(",")[2])))

In [8]:
HappinessdataSetRDDCountryScore.take(10)

[(1, 'Finland', 7.769),
 (2, 'Denmark', 7.6),
 (3, 'Norway', 7.554),
 (4, 'Iceland', 7.494),
 (5, 'Netherlands', 7.488),
 (6, 'Switzerland', 7.48),
 (7, 'Sweden', 7.343),
 (8, 'New Zealand', 7.307),
 (9, 'Canada', 7.278),
 (10, 'Austria', 7.246)]

In [9]:
## filtering data using filter function and maping as the (key, (value)) pair ## Transofmration
HappinessdataSetRDDCountryScorePak= HappinessdataSetRDDCountryScore.filter(lambda x: x[1]== "Pakistan").map(lambda y: (float(y[2]), y))

In [10]:
HappinessdataSetRDDCountryScorePak.take(10)

[(5.653, (67, 'Pakistan', 5.653))]

In [11]:
pakScore=HappinessdataSetRDDCountryScorePak.map(lambda x: x[1][2])

In [12]:
pakScore.take(10)

[5.653]

In [13]:
HappinessdataSetRDDCountryScore.map(lambda x: float(x[2])).take(10)

[7.769, 7.6, 7.554, 7.494, 7.488, 7.48, 7.343, 7.307, 7.278, 7.246]

In [14]:
filterData= HappinessdataSetRDDCountryScore.map(lambda x: (float(x[2]), x)).filter(lambda o: o[1][2] >= 5.653)

In [15]:
for dataset in filterData.collect(): print(dataset)

(7.769, (1, 'Finland', 7.769))
(7.6, (2, 'Denmark', 7.6))
(7.554, (3, 'Norway', 7.554))
(7.494, (4, 'Iceland', 7.494))
(7.488, (5, 'Netherlands', 7.488))
(7.48, (6, 'Switzerland', 7.48))
(7.343, (7, 'Sweden', 7.343))
(7.307, (8, 'New Zealand', 7.307))
(7.278, (9, 'Canada', 7.278))
(7.246, (10, 'Austria', 7.246))
(7.228, (11, 'Australia', 7.228))
(7.167, (12, 'Costa Rica', 7.167))
(7.139, (13, 'Israel', 7.139))
(7.09, (14, 'Luxembourg', 7.09))
(7.054, (15, 'United Kingdom', 7.054))
(7.021, (16, 'Ireland', 7.021))
(6.985, (17, 'Germany', 6.985))
(6.923, (18, 'Belgium', 6.923))
(6.892, (19, 'United States', 6.892))
(6.852, (20, 'Czech Republic', 6.852))
(6.825, (21, 'United Arab Emirates', 6.825))
(6.726, (22, 'Malta', 6.726))
(6.595, (23, 'Mexico', 6.595))
(6.592, (24, 'France', 6.592))
(6.446, (25, 'Taiwan', 6.446))
(6.444, (26, 'Chile', 6.444))
(6.436, (27, 'Guatemala', 6.436))
(6.375, (28, 'Saudi Arabia', 6.375))
(6.374, (29, 'Qatar', 6.374))
(6.354, (30, 'Spain', 6.354))
(6.321, (31,

In [16]:
filterData= HappinessdataSetRDDCountryScore.map(lambda x: (float(x[2]), x)).filter(lambda o: o[1][2] <= 5.653)

In [17]:
for dataset in filterData.collect(): print(dataset)

(5.653, (67, 'Pakistan', 5.653))
(5.648, (68, 'Russia', 5.648))
(5.631, (69, 'Philippines', 5.631))
(5.603, (70, 'Serbia', 5.603))
(5.529, (71, 'Moldova', 5.529))
(5.525, (72, 'Libya', 5.525))
(5.523, (73, 'Montenegro', 5.523))
(5.467, (74, 'Tajikistan', 5.467))
(5.432, (75, 'Croatia', 5.432))
(5.43, (76, 'Hong Kong', 5.43))
(5.425, (77, 'Dominican Republic', 5.425))
(5.386, (78, 'Bosnia and Herzegovina', 5.386))
(5.373, (79, 'Turkey', 5.373))
(5.339, (80, 'Malaysia', 5.339))
(5.323, (81, 'Belarus', 5.323))
(5.287, (82, 'Greece', 5.287))
(5.285, (83, 'Mongolia', 5.285))
(5.274, (84, 'North Macedonia', 5.274))
(5.265, (85, 'Nigeria', 5.265))
(5.261, (86, 'Kyrgyzstan', 5.261))
(5.247, (87, 'Turkmenistan', 5.247))
(5.211, (88, 'Algeria', 5.211))
(5.208, (89, 'Morocco', 5.208))
(5.208, (90, 'Azerbaijan', 5.208))
(5.197, (91, 'Lebanon', 5.197))
(5.192, (92, 'Indonesia', 5.192))
(5.191, (93, 'China', 5.191))
(5.175, (94, 'Vietnam', 5.175))
(5.082, (95, 'Bhutan', 5.082))
(5.044, (96, 'Cameroo

In [18]:
##HealthyLifeExpectency -- col 6

In [19]:
HappinessdataSetRDDHealthyLifeExpectency= HappinessdataSetRDD.map(lambda x: ((int(x.split(",")[0])), x.split(",")[1], float(float(x.split(",")[6])*100) ))

In [20]:
lifeHealthyRDD= HappinessdataSetRDDHealthyLifeExpectency.map(lambda x: (float(x[2]), x))

In [21]:
## use false for the descending order with sortbykey on RDD
lifeHealthyRDDSorted= lifeHealthyRDD.sortByKey(False) 

In [22]:
lifeHealthyRDD.take(10)

[(59.599999999999994, (1, 'Finland', 59.599999999999994)),
 (59.199999999999996, (2, 'Denmark', 59.199999999999996)),
 (60.3, (3, 'Norway', 60.3)),
 (59.099999999999994, (4, 'Iceland', 59.099999999999994)),
 (55.7, (5, 'Netherlands', 55.7)),
 (57.199999999999996, (6, 'Switzerland', 57.199999999999996)),
 (57.4, (7, 'Sweden', 57.4)),
 (58.5, (8, 'New Zealand', 58.5)),
 (58.4, (9, 'Canada', 58.4)),
 (53.2, (10, 'Austria', 53.2))]

In [23]:
##FreedomtoDecision -- col 7 in Dataset

In [33]:
HappinessdataSetRDDLifeFreedom= HappinessdataSetRDD.map(lambda x: ((int(x.split(",")[0])), x.split(",")[1], float(float(x.split(",")[7])*100) ))

In [34]:
HappinessdataSetRDDLifeFreedom.take(10)

[(1, 'Finland', 15.299999999999999),
 (2, 'Denmark', 25.2),
 (3, 'Norway', 27.1),
 (4, 'Iceland', 35.4),
 (5, 'Netherlands', 32.2),
 (6, 'Switzerland', 26.3),
 (7, 'Sweden', 26.700000000000003),
 (8, 'New Zealand', 33.0),
 (9, 'Canada', 28.499999999999996),
 (10, 'Austria', 24.4)]

In [35]:
lifeFreedomRDD= HappinessdataSetRDDLifeFreedom.map(lambda x: (float(x[2]), x))

In [36]:
lifeHealthyRDDSorted= lifeFreedomRDD.sortByKey(False) 

In [39]:
lifeHealthyRDDSorted.filter(lambda x: x[1][1]== "United Kingdom").take(10)

[(34.8, (15, 'United Kingdom', 34.8))]

In [53]:
## Sorting data based on the key value from the RDD of (Key, (value)) pair RDD
lifeHealthyRDDData= lifeHealthyRDDSorted.map(lambda x: (int(x[1][0]), x[1][1], float(x[1][2]) ))

In [54]:
for items in lifeHealthyRDDData.take(10): print(items)

(131, 'Myanmar', 56.599999999999994)
(92, 'Indonesia', 49.8)
(147, 'Haiti', 41.9)
(22, 'Malta', 37.5)
(121, 'Kenya', 37.2)
(95, 'Bhutan', 37.0)
(52, 'Thailand', 35.9)
(4, 'Iceland', 35.4)
(15, 'United Kingdom', 34.8)
(11, 'Australia', 33.2)


In [65]:
## converting the RDD to PySpark Dataframe and add column names as required using schema keyword
lifeHealthyRDDDataDf= lifeHealthyRDDData.toDF(schema=["id", "CountryName", "LifeFreedomPercent"])

In [66]:
## Show keyword is used to view PySpark Data
lifeHealthyRDDDataDf.show()

+---+--------------+------------------+
| id|   CountryName|LifeFreedomPercent|
+---+--------------+------------------+
|131|       Myanmar|56.599999999999994|
| 92|     Indonesia|              49.8|
|147|         Haiti|              41.9|
| 22|         Malta|              37.5|
|121|         Kenya|              37.2|
| 95|        Bhutan|              37.0|
| 52|      Thailand|              35.9|
|  4|       Iceland|              35.4|
| 15|United Kingdom|              34.8|
| 11|     Australia|              33.2|
|149|         Syria|              33.1|
|  8|   New Zealand|              33.0|
|  5|   Netherlands|              32.2|
| 41|    Uzbekistan|              32.2|
| 86|    Kyrgyzstan|              30.0|
| 16|       Ireland|29.799999999999997|
|  9|        Canada|28.499999999999996|
|100|         Nepal|28.499999999999996|
| 19| United States|28.000000000000004|
|153|      Tanzania|              27.6|
+---+--------------+------------------+
only showing top 20 rows



In [74]:
## Save it to csv by converting it to Pandas and using Pandas CSV option
lifeHealthyRDDDataDf.toPandas().to_csv('lifeHealthyDF.csv')